# Трансформеры
В этом домашнем задании мы рассмотим использование трансформеров в библиотеке PyTorch. Рассмотрим задачу языкового моделирования. Попробуем генерировать текст нейронной сетью. 

Ссылка на данные - https://drive.google.com/drive/folders/1x1A4ElliUGBPnHladGMwPxPuGxI8Vnpu?usp=sharing

In [ ]:
# хороший тон, импортировать все необходимые библиотеки в одной ячейке ;)

import torch
from torch import nn

import numpy as np
import time
import math
import codecs

!wget -O small_corp_for_test.txt https://drive.google.com/u/0/uc?id=1ijpKzdj4d1b0zYKot3_380OWtiI-2UJX&export=download

--2021-12-20 19:32:17--  https://drive.google.com/u/0/uc?id=1ijpKzdj4d1b0zYKot3_380OWtiI-2UJX
Resolving drive.google.com (drive.google.com)... 64.233.189.100, 64.233.189.102, 64.233.189.138, ...
Connecting to drive.google.com (drive.google.com)|64.233.189.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-54-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/3ge8qj133gdscoutdcn2mvhscd2aabbv/1640028675000/09709797766287458306/*/1ijpKzdj4d1b0zYKot3_380OWtiI-2UJX [following]
--2021-12-20 19:32:19--  https://doc-0s-54-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/3ge8qj133gdscoutdcn2mvhscd2aabbv/1640028675000/09709797766287458306/*/1ijpKzdj4d1b0zYKot3_380OWtiI-2UJX
Resolving doc-0s-54-docs.googleusercontent.com (doc-0s-54-docs.googleusercontent.com)... 74.125.203.132, 2404:6800:4008:c03::84
Connecting to doc-0s-54-docs.googleusercontent.com (doc-0s-54-docs.googleusercontent.com)|74

Что такое языковое моделирование? Это предсказание вероятности следующего токена (слова или буквы) на основе предыдущих токенов. Математически это можно описать так:

$$P(x_i|x_1, x_2 , ... , x_{i-1})$$ 

Последовательность $$ x_1, x_2, ... x_{i-1} $$ называют контекстом.

## Задание 0 (0 баллов, но сделать нужно)
Проставьте знаки неравенств, исходя из вашего опыта:
$$ P(раму | мама, мыла) > P(папу | мама, мыла) $$ 
(очевидно, вероятность второго события значительно реже, нежели первого)
$$ P(столу | дорога, ложка, к) < P(обеду | дорога, ложка, к) $$ 
(вторая последовательность является известной пословицей -> более вероятной последовательностью)
$$ P(Евпатий | меня, зовут) < P(Ваня | меня, зовут) $$ 
(Имя Ваня - более частое имя, чем Евпатий) 
$$ P(журналы | я, часто ,читаю) < P(комиксы | я, часто ,читаю) $$ 
(из личного опыта))))
Попробуйте объяснить выбор для каждого из примеров.

Ответ : 

Если для некоторых из примеров проставить знаки достаточно просто, то на некоторые сложно сказать, какой овтет верный. Мы принимаем решение для данного задания исходя их опыта использования русского языка. Мы много читали на русском и слушали огромное количество русской речи. Обучение языковых моделей происходит по схожему принципу. 

Мы хотим показать модели столько текстов, сколько можем и надеемся, что она наберется достаточно опыта, чтобы расставлять такие знаки неравества максимально схоже с человеком.

## Задание 1 (0.5 балла)
Мы будем обучать языковую модель для предсказания следущей буквы. Такие языковые модели применяются в распозновании речи, так как предоставляют дополнительную информацию акустической модели при выборе следующего символа. Для начала, откройте файл с данными, посмотрите, какие символы входят в тексты, сколько их. Уберите из текста все символы переноса на новую строку и табуляцию.

In [ ]:
path = 'small_corp_for_test.txt'
file = codecs.open(path, 'r', "utf-8")
data = file.readlines()
file.close()
len(data)

700000

In [ ]:
# YOUR CODE HERE

for i in range(len(data)):
    data[i] = data[i].replace('\n', '')
    data[i] = data[i].replace('\t', '')
data[5]

################

'что происходит'

## Задание 2 (0.5 балла)
Для обучения модели требуется сначала подготовить текст в подходящий для нейросети вид. Важно также отметить, что нужно добавить два токена start и end, которые отвечают за начало и конец текста. Используйте [ и ] для этой задачи. Также нам нужен токен pad, чтобы заполнять им текст до требуемой длинны для формирования батча.

Реализуйте метод preprocess класса Preprocessor. Он должен принимать на вход текст и длинну текста, которую мы ожидаем получить на выходе. Текст должен быть переведен в нижний регистр, в конец текста добавляется требуемое число pad токенов, далее текст векторизуется (каждому символу ставится свое число). Вернуть требуется два вектора. Полученный результат без последнего токена (на нем будем обучаться) и полученный результат без первого токена (целевые метки при обучении).

In [ ]:
class Preprocessor:
    def __init__(self):
        self.alphabet = '_добсркгаупитнезчмф яжлйвцыэь-шхющёъ][ '
        self.token2ind = {}
        self.ind2token = {}
        for i in range(len(self.alphabet)):
            self.token2ind[self.alphabet[i]] = i
            self.ind2token[i] = self.alphabet[i]
        
    
    def preprocess(self, text, window_size):
        # YOUR CODE HERE
        v = np.zeros(window_size, dtype=np.longlong)
        s = text
        while len(s) < window_size:
            s += '_'
        for i in range(window_size):
            v[i] = self.token2ind[s[i]]
        return v[:-1], v[1:]

## Задание 3 (0.5 балла)
Так как мы решили, что текст будет начинаться токеном [ и заканчиваться токеном ], данные нужно поправить. Реализуйте эту идею, добавьте данные токены в ваши тексты.

In [ ]:
# YOUR CODE HERE
for i in range(len(data)):
    data[i] = '[' + data[i] + ']'
data[5]

'[что происходит]'

## Задание 4 (0.5 балла)
Так как мы не располагаем большими мощностями, то давайте ограничим максимальную длинну текста. Вы можете менять этот порог и тем самым уменьшать кол-во текстов в вашей выборке и увеличивая тем самым скорость обучения. Начнем же мы с 128. 
Выберите порог и оставьте только те тексты, длина которых не превосходит данный порог.

Далее разбейте тексты на train и test, перемешайте тексты при разбиении, размер тестовой выборки должен быть 15% от общего числа текстов. 

In [ ]:
list(range(5,-1,-1))

[5, 4, 3, 2, 1, 0]

In [ ]:
THRESHOLD = 128

# YOUR CODE HERE
for i in range(len(data)-1, -1, -1):
    if len(data[i]) > THRESHOLD:
        data.pop(i)

In [ ]:
from sklearn.model_selection import train_test_split

data_train, data_test = train_test_split(data, test_size=0.15, shuffle=True,random_state=100)

In [ ]:
print(len(data), len(data_test) / len(data_train))

683438 0.17647119578876339


## Задание 5 (2 балла)
Напишем датасет. На вход датасету передается набор текстов, объект класса Preprocessor и размер окна, который вы выбрали в прошлом задании.
Реализуйте методы __len__ и __getitem__.

In [ ]:
class TextDataset(torch.utils.data.Dataset):
    
    def __init__(self, x, preproc, win_size = 128):
        # YOUR CODE HERE
        self.x = x
        self.preproc = preproc
        self.win_size = win_size
        self.length = len(x)
        ################
    
    def __len__(self):
        # YOUR CODE HERE
        return self.length
        ################
    
    def __getitem__(self, idx):
        # YOUR CODE HERE
        return self.preproc.preprocess(self.x[idx], self.win_size)
        ################

In [ ]:
preproc = Preprocessor()
train_dataset = TextDataset(data_train, preproc)
test_dataset = TextDataset(data_test, preproc)

## Задание 6 (2 балла)
Напишем модель. Класс для реализации positional encoding реализован за вас, он нужен, чтобы модель могла после получения эмбедингов понимать, на каком месте какой токен находится.

Заполните пропуски в классе модели. Гипперпараметры модели вам предлагается подобрать самостоятельно. Рекомендуется использовать не более 6 слоев в трансформере. В декореде испоьлзуйте две линейных слоя с функцией активации ReLU между ними.

## Задание 6_1 (0 баллов, но надо ответить!)
При обучении языковой модели на основе трансформеров мы используем маскирование символов (как мы это делаем - уже реализовано). Напишите, почему мы это делаем? Почему это так важно?

Ответ:

Это необходимо в слоях самовнимания в кодировщике - чтобы они не могли "посмотреть" на последующие токены

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=500):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

### (Взял многое из Pytorch-Tutorial, чтобы разобраться с возможными начальными коэффициентами)

In [ ]:
class LanguageModel(nn.Module):
    def __init__(self, vocab_size: int, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super(LanguageModel, self).__init__()
        self.emb = nn.Embedding(vocab_size, d_model)
        self.pe = PositionalEncoding(d_model, dropout)
        self.transformer_encoder_layer = nn.TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_encoder_layer, nlayers)
        self.decoder = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU(),
            nn.Linear(d_model, vocab_size))
    
    def forward(self, x, src_mask):
        x = self.emb(x)
        x = self.pe(x)
        x = x.transpose(1, 0)
        x = self.transformer_encoder(x, src_mask)
        x = self.decoder(x)
        return x.transpose(1, 0)
    
    def generate_square_subsequent_mask(self, sz):
        # А вот и то самое маскирование
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

In [ ]:
model = LanguageModel(len('_добсркгаупитнезчмф яжлйвцыэь-шхющёъ][ '), 200, 2, 200, 2, 0.2)

## Задание 7 (2,5 балла)
Финишная прямая. Давайте реализуем класс для обучения модели и ее валидации. Следуйте указаниям в коде и заполните недостающие фрагменты в коде.

In [ ]:
class Trainer:
    
    def __init__(self, model, train_dataset, test_dataset):
        
        self.model = model
        self.ntokens = len('_добсркгаупитнезчмф яжлйвцыэь-шхющёъ][ ')
        
        self.train_batch_size = 127
        self.test_batch_size = 127
        
        self.train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=self.train_batch_size, shuffle=True)
        self.test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=self.test_batch_size, shuffle=True)
        self.train_dataloader_size = train_dataset.__len__() // self.train_batch_size
        self.test_dataloader_size = test_dataset.__len__() // self.test_batch_size
        
        self.device = 'cuda:0'
        self.criterion = nn.CrossEntropyLoss(ignore_index=0) # используйте CrossEntrophyLoss, передайте в качетсве параметра 
                             # ignore index индекс символа _, чтобы модель не штрафовалась за то
                             # что идет после закрывающего токена
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)
        self.steps_to_print = 100
        
    def train_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        src_mask = self.model.generate_square_subsequent_mask(127).to(self.device)
        for batch in self.train_dataloader:
            inputs, targets = batch
            # YOUR CODE HERE
            
            # реализуйте шаги обучения модели
            # сохраняйте значение ошибки в переменную counted_loss
            step += 1
            it += 1
            inputs, targets = torch.LongTensor(inputs), torch.LongTensor(targets)
            inputs, targets = inputs.to(self.device), targets.to(self.device)
            results = self.model(inputs, src_mask)
            loss = self.criterion(results.transpose(1,2), targets)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            counted_loss += loss.item()
            ################
            
            
            if step%self.steps_to_print == 0:
                result = 'Train epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.train_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
    
    def validate_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        src_mask = self.model.generate_square_subsequent_mask(127).to(self.device)
        for batch in self.test_dataloader:
            inputs, targets = batch
            
            # YOUR CODE HERE
            
            # реализуйте шаги для теста модели
            # помните, что данный метод уже запускается из 
            # блока with torch.no_grad(), а потому 
            # повторно его использовать не нужно
            step += 1
            it += 1
            inputs, targets = torch.LongTensor(inputs), torch.LongTensor(targets)
            inputs, targets = inputs.to(self.device), targets.to(self.device)
            results = self.model(inputs, src_mask)
            loss = self.criterion(results.transpose(1,2), targets)
            counted_loss += loss.item()
            
            ################
            
            if step%(self.steps_to_print//2) == 0:
                result = 'Validate epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.test_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
        
    def train(self, number_of_epochs):
        model.to(self.device)
        for epoch in range(1, number_of_epochs+1):
            model.train()
            #self.train_one_epoch(epoch)
            with torch.no_grad():
                model.eval()
                self.validate_one_epoch(epoch)
            print()

In [ ]:
train_dataset.__len__()

580922

Что такое ppl? Перплексия. Ее можно интерпретировать как меру "удивленности" модели нужному символу. Чем меньше данная величина, тем лучше, ведь это значит, что модель если и сделала неправильный выбор, то не сильно удивлена своей ошибке.

Проведите несколько экспериментов, посмотрите, при каких гипперпараметрах значение перплексии минимально.

## Задание 8 (0.5 балла)
Запустите обучение на нескольких эпохах. Ориентируйтесь на ваши вычислительные мощности и время работы. Вы всегда можете посчитать, сколько секунд уходит на один батч.

In [ ]:
# YOUR CODE HERE
###############

tr = Trainer(model,train_dataset, test_dataset)
tr.train(1)

Validate epoch 1 | Step 50/807 | Counted loss 98.04364037513733 | ppl 7.105526115979055 | time 2.049182176589966 | 
Validate epoch 1 | Step 100/807 | Counted loss 98.09216642379761 | ppl 7.112425525569575 | time 1.8907289505004883 | 
Validate epoch 1 | Step 150/807 | Counted loss 97.6574672460556 | ppl 7.050858235318007 | time 1.8789403438568115 | 
Validate epoch 1 | Step 200/807 | Counted loss 98.33454871177673 | ppl 7.1469877500360095 | time 1.883697509765625 | 
Validate epoch 1 | Step 250/807 | Counted loss 98.21051979064941 | ppl 7.129281056936987 | time 1.8582439422607422 | 
Validate epoch 1 | Step 300/807 | Counted loss 97.92279601097107 | ppl 7.088373596470628 | time 1.8804700374603271 | 
Validate epoch 1 | Step 350/807 | Counted loss 98.4006427526474 | ppl 7.1564414630093935 | time 1.8762269020080566 | 
Validate epoch 1 | Step 400/807 | Counted loss 98.73631691932678 | ppl 7.204647748488083 | time 1.8924622535705566 | 
Validate epoch 1 | Step 450/807 | Counted loss 97.982587814

## Задание 9 (1 балл)
Итак, давайте попробуем погенерировать текст нашей сеткой. Закончите функцию по генерации текста. Попробуйте сгенерировать какой-нибудь текст. Помните, что если вы хотите генерировать текст с нуля, то вы должны передать в качестве текста только токен start.
Прекратите генерировать текст, если модель выдала токен end или длинна текста больше 150.

In [ ]:
def generate_text(text):
    x = []
    src_mask = model.generate_square_subsequent_mask(127).to('cpu')
    for letter in text:
        x.append(preproc.token2ind[letter])
    x = torch.from_numpy(np.array(x))
    
    pred = model(x, src_mask)
    return pred
    ind = ... 
    
    text += ... 
    
    if len(text) > 150 or ind == 36:
        return text
    else:
        return generate_text(text)

In [ ]:
generate_text('я')

RuntimeError: ignored

## Задание 10* (Задание - бонус, 5 баллов за реализацию при условии, что сделаны прошлые задания)
Давайте вспомним, что такое transfer learning. Мы хотим использовать уже предобученные эмбединги для нашей сети, чтобы наша сеть обучалась быстрее. Давайте попробуем обучить новую модель на уровне слов, а не символов, но для упрощения задачи используем предобученный слой из библиотеки Natasha, а вернее, ее блок Navec.

[Изучите](https://github.com/natasha/navec) то, как вставить слой в вашу нейронную сеть.

Теперь мы хотим, чтобы на вход модели подавались слова, модифицируйте ваш датасет. Возвращайте теперь номер слова в словаре navec.

In [ ]:
class TextDataset_Navec(torch.utils.data.Dataset):
    
    def __init__(self, x, win_size = 128):
        # YOUR CODE HERE
        self.navec = ...
        ################
    
    def __len__(self):
        # YOUR CODE HERE
        ################
    
    def __getitem__(self, idx):
        # YOUR CODE HERE
        ################

IndentationError: expected an indented block (<ipython-input-30-84de98a61cab>, line 12)

Немного модифицируем модель. Теперь нам не нужны слои с трансформером, так как весь механизм внимания уже заложен в ембедингах. Давайте попробуем просто пройтись линейной головой над эмбедингами. Выберите параметры самостоятельно.

In [ ]:
class LanguageModel(nn.Module):
    def __init__(self):
        super(LanguageModel, self).__init__()
        self.emb_navec = ...
        self.head = ...
    
    def forward(self, x):
        x = ... # emb
        x = ... # head
        return x

Теперь дело за малым! Надо немного модифицировать класс обучения, так как мы не используем маскирование, после чего можно приступить к тесту!

In [ ]:
class Trainer:
    
    def __init__(self, model, train_dataset, test_dataset):
        
        self.model = model
        
        self.train_batch_size = 64
        self.test_batch_size = 64
        
        self.train_dataloader = ...
        self.test_dataloader = ...
        self.train_dataloader_size = ...
        self.test_dataloader_size = ...
        
        self.device = 'cuda:0'
        self.criterion = ... 
        
        self.optimizer = ...
        
        self.steps_to_print = 1000
        
    def train_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        
        for batch in self.train_dataloader:
            x, y = batch
            # YOUR CODE HERE
            
            # реализуйте шаги обучения модели
            # сохраняйте значение ошибки в переменную counted_loss
            
            ################
            
            
            if step%self.steps_to_print == 0:
                result = 'Train epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.train_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
    
    def validate_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        for batch in self.test_dataloader:
            x, y = batch
            
            # YOUR CODE HERE
            
            # реализуйте шаги для теста модели
            # помните, что данный метод уже запускается из 
            # блока with torch.no_grad(), а потому 
            # повторно его использовать не нужно
            
            ################
            
            if step%(self.steps_to_print//2) == 0:
                result = 'Validate epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.test_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
        
    def train(self, number_of_epochs):
        model.to(self.device)
        for epoch in range(1, number_of_epochs+1):
            model.train()
            self.train_one_epoch(epoch)
            with torch.no_grad():
                model.eval()
                self.validate_one_epoch(epoch)
            print()

Запустите обучение. 

In [ ]:
# YOUR CODE HERE
###############